# Preparation
net 6 SDK installieren

In [ ]:
dotnet tool install --global dotnet-ef                              

Das Tool "dotnet-ef" ist bereits installiert.
System.IO.FileNotFoundException: Could not find a MSBuild project file in 'D:\Projekte\dartsEF'. Specify which project to use with the --project option.
   at Microsoft.Extensions.SecretManager.Tools.Internal.MsBuildProjectFinder.FindMsBuildProject(String project)
   at Microsoft.Extensions.SecretManager.Tools.Internal.InitCommand.ResolveProjectPath(String name, String path)
   at Microsoft.Extensions.SecretManager.Tools.Internal.InitCommand.Execute(CommandContext context)
   at Microsoft.Extensions.SecretManager.Tools.Internal.InitCommand.Execute(CommandContext context, String workingDirectory)
   at Microsoft.Extensions.SecretManager.Tools.Internal.InitCommandFactory.Execute(CommandContext context, String workingDirectory)
   at Microsoft.Extensions.SecretManager.Tools.Program.RunInternal(String[] args)
   at Microsoft.Extensions.SecretManager.Tools.Program.TryRun(String[] args, Int32& returnCode)
Command failed : Could not find a MSBuild

# neue Dart Solution

In [ ]:
mkdir darts
cd darts
git init
dotnet new gitignore
dotnet new sln

# neues Dart Project

In [ ]:
dotnet new console -o .\EFmanymanymany
dotnet sln add .\EFmanymanymany
cd .\EFmanymanymany
dotnet user-secrets init
$projDir = pwd

# up and running

In [ ]:
git add *
git commit -m "initial"

# DB Context generieren

In [ ]:
$projDir = "D:\Projekte\dartsEF\EFmanymanymany"

In [ ]:
pushd $projDir
dotnet add package Microsoft.EntityFrameworkCore.Design 
dotnet add package Microsoft.EntityFrameworkCore.SqlServer
dotnet add package Microsoft.Extensions.Configuration.Json
dotnet add package Microsoft.Extensions.Configuration.Binder

## Fail 1
```pwsh
dotnet ef dbcontext scaffold Context=DartsContext2 'Data Source=(localdb)\batmanDB;Initial Catalog=darts' Microsoft.EntityFrameworkCore.SqlServer
```
Mein generierter Context mit integratedAuthentication bringt "Login failed". Daher setze ich ein Passwort für SA

```sql
USE [master]
GO
ALTER LOGIN [sa] WITH PASSWORD=N'Start123'
GO
ALTER LOGIN [sa] ENABLE
GO
```

In [ ]:
pushd $projDir
dotnet ef dbcontext scaffold 'Data Source=(localdb)\batmanDB;Initial Catalog=darts;User ID=sa;Password=Start123' Microsoft.EntityFrameworkCore.SqlServer -c "Darts2Context" -o .\Models

In [ ]:

#dotnet user-secrets set ConnectionStrings:DartsDB "Data Source=(localdb)\batmanDB;Initial Catalog=darts;"
#dotnet ef dbcontext scaffold Context=DartsContext2 Name=DartsDB Microsoft.EntityFrameworkCore.SqlServer
dotnet ef dbcontext scaffold Context=DartsContext2 'Data Source=(localdb)\batmanDB;Initial Catalog=darts;User ID=sa;Password=Start123' Microsoft.EntityFrameworkCore.SqlServer

## Fail 2
Die Verwendung von user-secrets scheint mit scaffolding noch nicht zu funktionieren

```pwsh
dotnet user-secrets set ConnectionStrings:DartsDB "Data Source=(localdb)\batmanDB;Initial Catalog=darts;"
dotnet ef dbcontext scaffold Name=DartsDB Microsoft.EntityFrameworkCore.SqlServer -c Darts2Context
```

In [ ]:
git add *
git commit -m "add db context"

* add some models
* dont forget to add them as DBSet in DartsContext

In [ ]:
dotnet ef migrations add "initial"
dotnet ef database update

In [ ]:
pushd $projDir
dotnet run


Unhandled exception. Microsoft.Data.SqlClient.SqlException (0x80131904): There is already an object named 'Games' in the database.
   at Microsoft.Data.SqlClient.SqlConnection.OnError(SqlException exception, Boolean breakConnection, Action`1 wrapCloseInAction)
   at Microsoft.Data.SqlClient.SqlInternalConnection.OnError(SqlException exception, Boolean breakConnection, Action`1 wrapCloseInAction)
   at Microsoft.Data.SqlClient.TdsParser.ThrowExceptionAndWarning(TdsParserStateObject stateObj, Boolean callerHasConnectionLock, Boolean asyncClose)
   at Microsoft.Data.SqlClient.TdsParser.TryRun(RunBehavior runBehavior, SqlCommand cmdHandler, SqlDataReader dataStream, BulkCopySimpleResultSet bulkCopyHandler, TdsParserStateObject stateObj, Boolean& dataReady)
   at Microsoft.Data.SqlClient.SqlCommand.RunExecuteNonQueryTds(String methodName, Boolean isAsync, Int32 timeout, Boolean asyncWrite)
   at Microsoft.Data.SqlClient.SqlCommand.InternalExecuteNonQuery(TaskCompletionSource`1 completion, B

In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer,*-*"
#!connect mssql -h


Installed Packages Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.21606.1

mssql
  Connects to a Microsoft SQL Server database

Usage:
  [options] #!connect mssql <connectionString>

Arguments:
  <connectionString>  The connection string used to connect to the database

Options:
  --create-dbcontext           Scaffold a DbContext in the C# kernel.
  --kernel-name <kernel-name>  The name of the subkernel to be added
  -?, -h, --help               Show help and usage information

